## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP1 - Verificación de la calidad de los datos

---

En esta libreta se realiza una verificación de calidad de datos con base en los planteado en el Anexo SP4. 

---

In [4]:
#Se importan las librerias a utilizar

import pandas as pd

/home/sebastian/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


----

##### Lectura de los datasets

In [5]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_vc.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_vc.csv", sep=";")
display(df_tarjetas.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0


---
#### Verificación de calidad de datos

**Análisis a realizar**

1. Evaluación de valores nulos (filas y columnas)
2. Evaluación de formato válido
3. Valores ajustados en rangos (ver anexos)
4. Claves únicas
5. Integridad referencial
6. Cumplimiento de reglas en valores

In [6]:
# Establecimiento de los umbrales de aceptación

FORMATEO_VALORES = 0.1
RANGOS_VALORES = 0.0
INTEGRIDAD_REF = 0.1
REGLAS_VALORES = 0.1

# Valores globales

cantidad_filas_creditos = df_creditos.shape[0]
cantidad_filas_tarjetas = df_tarjetas.shape[0]

## Dimensión: completitud

### (1a) Filas

In [7]:
# Se obtienen las cantidades de valores nulos por columna

nulos_x_columna = df_creditos.isna().sum()

print(f"Cantidad de filas que tienen valores nulos por atributo:\n{nulos_x_columna}")

Cantidad de filas que tienen valores nulos por atributo:
id_cliente               0
edad                     0
importe_solicitado       0
duracion_credito         0
antiguedad_empleado    337
situacion_vivienda       0
ingresos                 0
objetivo_credito         0
pct_ingreso              0
tasa_interes           912
estado_credito           0
falta_pago               0
dtype: int64


In [8]:
# De cualquier manera se establece el cálculo a realizar

cantidad_columnas = len(df_creditos.axes[1])

df_creditos['completitud_fila'] = (df_creditos.isnull().sum(axis=1) / cantidad_columnas)

problemas = df_creditos[df_creditos['completitud_fila'] >= 0.2]

completitud_f = problemas.shape[0]

print(f"Filas que incumplen el umbral de nulos en columnas [completitud_f]:")
print(f"{completitud_f} ({round((completitud_f  / cantidad_filas_creditos) * 100, 2)})%")

Filas que incumplen el umbral de nulos en columnas [completitud_f]:
0 (0.0)%


### (1b) Dataset

In [9]:
completitud_d = df_creditos.isnull().any(axis=1).sum()

print(f"Filas que presentan nulos en el dataset [completitud_d]:")
print(f"{completitud_d} ({round((completitud_d  / cantidad_filas_creditos) * 100, 2)})%")

Filas que presentan nulos en el dataset [completitud_d]:
1225 (12.1)%


----

## Dimensión: exactitud

### (2) Formato válido

In [10]:
# No se encuentran atributos con formato específico

IndexError: list index out of range

----

## Dimensión: exactitud

### (3) Valores ajustados

Atributo: **edad**

In [11]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: edad

valores = pd.value_counts(df_creditos['edad']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['edad']) - df_creditos['edad'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['edad'] > 90]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

/tmp/xpython_151732/2105537402.py:5: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.


Distribución inicial del atributo: 
edad
22     2228
23     2190
24     1926
25     1646
26     1328
21      799
20        6
144       2
123       2
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila
81,715623483.0,144,4800,3,4.0,ALQUILER,250000,INVERSIONES,0.02,13.57,0,N,0.000000
183,815500983.0,144,6000,2,4.0,HIPOTECA,200000,EDUCACIÓN,0.03,11.86,0,N,0.000000
575,717956283.0,123,20400,3,2.0,ALQUILER,80004,EDUCACIÓN,0.25,10.25,0,N,0.000000
747,712881483.0,123,20000,4,7.0,ALQUILER,78000,INVERSIONES,0.26,NaN,0,N,0.083333


Cantidad detectada: 4


In [12]:
def calcular_rangos_valores_edad():
    edad_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo edad: {edad_valores_fuera_rango}")
    
    indicador = (edad_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo edad): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_edad()

Cantidad de filas con valores fuera de rango en atributo edad: 4
Porcentaje de filas con errores de rango de valores (atributo edad): 0.04 %
Evaluación: no cumplimiento


## Atributo antiguedad empleado

In [13]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: edad

valores = pd.value_counts(df_creditos['antiguedad_empleado']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['antiguedad_empleado']) - df_creditos['antiguedad_empleado'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['antiguedad_empleado'] > 13]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

/tmp/xpython_151732/3409525708.py:5: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.


Distribución inicial del atributo: 
antiguedad_empleado
0.0      1414
2.0      1281
3.0      1128
5.0      1050
6.0      1027
1.0       993
4.0       846
7.0       784
8.0       587
9.0       414
10.0      237
11.0       27
123.0       2
Name: count, dtype: int64

Cantidad de nulos en el atributo: 337

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y,0.0
210,715999608.0,21,20000,4,123.0,HIPOTECA,192000,INVERSIONES,0.10,6.54,0,N,0.0


Cantidad detectada: 2


In [14]:
def calcular_rangos_valores_edad():
    edad_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo antiguedad empleado: {edad_valores_fuera_rango}")
    
    indicador = (edad_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo edad): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_edad()

Cantidad de filas con valores fuera de rango en atributo antiguedad empleado: 339
Porcentaje de filas con errores de rango de valores (atributo edad): 3.35 %
Evaluación: no cumplimiento


Se procesa el atributo: **situacion_vivienda**


In [15]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: situacion_vivienda

valores = pd.value_counts(df_creditos['situacion_vivienda']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['situacion_vivienda']) - df_creditos['situacion_vivienda'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'ALQUILER|PROPIA|HIPOTECA|OTROS' # Se define una re de los valores validos según el anexo

df_creditos['situacion_vivienda_ok'] = df_creditos['situacion_vivienda'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_creditos[df_creditos['situacion_vivienda_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['situacion_vivienda_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

/tmp/xpython_151732/504200136.py:5: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.


Distribución inicial del atributo: 
situacion_vivienda
ALQUILER    6125
HIPOTECA    3223
PROPIA       741
OTROS         38
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok


Cantidad detectada: 0


In [16]:
def calcular_rangos_valores_situacion_vivienda():
    situacion_vivienda_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo situacion_vivienda: {situacion_vivienda_valores_fuera_rango}")
    
    indicador = (situacion_vivienda_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo situacion_vivienda): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_situacion_vivienda()

Cantidad de filas con valores fuera de rango en atributo situacion_vivienda: 0
Porcentaje de filas con errores de rango de valores (atributo situacion_vivienda): 0.0 %
Evaluación: ok


## Atributo objetivo_credito

In [17]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: situacion_vivienda

valores = pd.value_counts(df_creditos['objetivo_credito']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['objetivo_credito']) - df_creditos['objetivo_credito'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'ALQUILER|PROPIA|HIPOTECA|OTROS' # Se define una re de los valores validos según el anexo

df_creditos['objetivo_credito'] = df_creditos['objetivo_credito'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_creditos[df_creditos['objetivo_credito'] == 'EDUCACIÓN']) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['situacion_vivienda_ok'] == 'EDUCACIÓN'] 
print(f"Cantidad detectada: {resultado.shape[0]}")

/tmp/xpython_151732/1227465492.py:5: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.


Distribución inicial del atributo: 
objetivo_credito
EDUCACIÓN        2328
SALUD            1853
INVERSIONES      1753
PAGO_DEUDAS      1673
PERSONAL         1643
MEJORAS_HOGAR     877
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok


Cantidad detectada: 0


In [18]:
def calcular_rangos_valores_situacion_vivienda():
    situacion_vivienda_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo objetivo_credito: {situacion_vivienda_valores_fuera_rango}")
    
    indicador = (situacion_vivienda_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo situacion_vivienda): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_situacion_vivienda()

Cantidad de filas con valores fuera de rango en atributo objetivo_credito: 0
Porcentaje de filas con errores de rango de valores (atributo situacion_vivienda): 0.0 %
Evaluación: ok


## atributo importe_solicitado

In [19]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: edad

valores = pd.value_counts(df_creditos['importe_solicitado']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['importe_solicitado']) - df_creditos['importe_solicitado'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['importe_solicitado'] > 110000]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

/tmp/xpython_151732/2692523258.py:5: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.


Distribución inicial del atributo: 
importe_solicitado
5000     800
6000     715
8000     556
4000     446
7000     436
        ... 
750        1
700        1
19800      1
4150       1
9425       1
Name: count, Length: 509, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok


Cantidad detectada: 0


In [20]:
def calcular_rangos_valores_situacion_vivienda():
    situacion_vivienda_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo importe_solicitado: {situacion_vivienda_valores_fuera_rango}")
    
    indicador = (situacion_vivienda_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo situacion_vivienda): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_situacion_vivienda()

Cantidad de filas con valores fuera de rango en atributo importe_solicitado: 0
Porcentaje de filas con errores de rango de valores (atributo situacion_vivienda): 0.0 %
Evaluación: ok


## Atributo duracion credito

In [21]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: edad

valores = pd.value_counts(df_creditos['duracion_credito']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['duracion_credito']) - df_creditos['duracion_credito'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['duracion_credito'] > 3]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
duracion_credito
2    3404
3    3364
4    3359
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok
4,710821833.0,24,35000,4,8.0,ALQUILER,54400,False,0.55,14.27,1,Y,0.0,True
7,810347208.0,24,35000,4,5.0,ALQUILER,78956,False,0.44,11.11,1,N,0.0,True
10,719661558.0,22,35000,4,6.0,ALQUILER,85000,False,0.41,10.37,1,N,0.0,True
13,816082233.0,26,35000,4,4.0,ALQUILER,108160,False,0.32,18.39,1,N,0.0,True
14,712396908.0,23,35000,4,2.0,ALQUILER,115000,False,0.30,7.90,0,N,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10114,712210458.0,25,9500,4,2.0,ALQUILER,57000,False,0.17,6.17,0,N,0.0,True
10115,714109308.0,25,9500,4,1.0,ALQUILER,57000,False,0.17,12.53,0,N,0.0,True
10121,708564858.0,22,9500,4,1.0,ALQUILER,69000,False,0.14,15.33,1,Y,0.0,True
10122,713755458.0,25,7000,4,9.0,PROPIA,65000,False,0.11,11.66,0,Y,0.0,True


Cantidad detectada: 3359


In [22]:
def calcular_rangos_valores_edad():
    edad_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo duracion_credito: {edad_valores_fuera_rango}")
    
    indicador = (edad_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo edad): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_edad()

Cantidad de filas con valores fuera de rango en atributo duracion_credito: 3359
Porcentaje de filas con errores de rango de valores (atributo edad): 33.17 %
Evaluación: no cumplimiento


Dataset: **datos_tarjetas**

In [23]:
# Se obtiene el valor de la cantidad de filas actual
cant_antes = df_tarjetas.shape[0]  

# Se ordena el dataset según el atributo que se desee evaluar (requerido para el paso siguiente)
df_tarjetas.sort_values("id_cliente", inplace=True)

# Se detectan y eliminan los duplicados en un atributo dejando la última ocurrencia
df_tarjetas.drop_duplicates(subset ="id_cliente", keep = 'last', inplace = True)

# Se obtiene el valor posterior a la operación
cant_despues = df_tarjetas.shape[0]

#Se imprimen ambos valores
print('Dataset: tarjetas')
print(f"Antes del análisis de duplicados: {cant_antes} - Despues del filtrado de duplicados: {cant_despues}")
if cant_antes > cant_despues:
    diferencia = cant_antes - cant_despues
    pct_diferencia = ((cant_antes - cant_despues) / cant_antes) * 100
    print(f"Se detectaron claves duplicadas en {diferencia} fila(s) un {round(pct_diferencia, 2)}%.")
else:
    print("No se detectaron claves duplicadas")

Dataset: tarjetas
Antes del análisis de duplicados: 10127 - Despues del filtrado de duplicados: 10127
No se detectaron claves duplicadas


### (5) Integridad referencial

In [24]:
# Las uniones se hacen de a pares - revisar nombres de atributos

df_integrado = pd.merge(df_creditos, df_tarjetas, on='id_cliente', how='inner')
coincidencias = df_integrado.shape[0]

print(f"Datos de créditos: {cantidad_filas_creditos} - Coincidencias con datos de tarjetas: {coincidencias}")

print("\nSe visualiza el dataset resultante:")
display(df_integrado.head(5))

print(f"Reporte general:\n \
- Filas del dataset creditos (inicial): {cantidad_filas_creditos}\n \
- Filas del dataset tarjetas (inicial): {cantidad_filas_tarjetas}\n \
- Errores detectados en la operación de unión: {abs(coincidencias - cantidad_filas_creditos)} \n \
- Filas del dataset unificado: {df_integrado.shape[0]}")

Datos de créditos: 10127 - Coincidencias con datos de tarjetas: 10127

Se visualiza el dataset resultante:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,False,0.59,16.02,1,Y,0.0,True,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0
1,768805383.0,21,1000,2,5.0,PROPIA,9600,False,0.10,11.14,0,N,0.0,True,39.0,CASADO,ACTIVO,1144.0,M,12691.0,SECUNDARIO_COMPLETO,Blue,42.0,3.0
2,818770008.0,25,5500,3,1.0,HIPOTECA,9600,False,0.57,12.87,1,N,0.0,True,44.0,SOLTERO,ACTIVO,1291.0,F,8256.0,UNIVERSITARIO_COMPLETO,Blue,33.0,5.0
3,713982108.0,23,35000,2,4.0,ALQUILER,65500,False,0.53,15.23,1,N,0.0,True,36.0,CASADO,ACTIVO,1887.0,M,3418.0,UNIVERSITARIO_COMPLETO,Blue,20.0,3.0
4,710821833.0,24,35000,4,8.0,ALQUILER,54400,False,0.55,14.27,1,Y,0.0,True,54.0,CASADO,ACTIVO,1314.0,M,9095.0,DESCONOCIDO,Blue,26.0,1.0


Reporte general:
 - Filas del dataset creditos (inicial): 10127
 - Filas del dataset tarjetas (inicial): 10127
 - Errores detectados en la operación de unión: 0 
 - Filas del dataset unificado: 10127


In [25]:
def calcular_integridad_referencial():
    cant_problemas = cantidad_filas_creditos - df_integrado.shape[0] # Se calcula sobre el inicio (foco)
    print(f"Casos de problemas de integridad referencial: {cant_problemas}")

    indicador = (cant_problemas / cantidad_filas_creditos)
    print(f"Porcentaje de filas con problemas de integridad referencial: {round(indicador * 100, 2)} %")

    if (indicador > INTEGRIDAD_REF):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')
        
calcular_integridad_referencial()

Casos de problemas de integridad referencial: 0
Porcentaje de filas con problemas de integridad referencial: 0.0 %
Evaluación: ok


## Atributo estado_civil

---

## Dimensión exactitud (bis)
### (6) Reglas en valores

Regla 1: Para aquellos casos en que los créditos constituyan un porcentaje de los ingresos del cliente mayor al 50% sus ingresos deberán ser mayores a 20.000.

Regla 2: Para aquellos créditos cuya duración sea la mínima permitida el porcentaje de los ingresos del cliente (con respecto al importe solicitado) no podrá exceder el 60% salvo en los casos en los que sea propietario de su vivienda.

In [26]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_integrado.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_resultado = pd.concat([df_integrado, regla_pct_ingresos], axis=1)  

# Se visualizan los datos
print("Se visualizan las tuplas que no cumplen con la regla:\n")
display(df_resultado[df_resultado.regla_pct_ingresos == 'err'].head())


# Se verifica la cantidad de elementos
aux = df_resultado[df_resultado.regla_pct_ingresos == 'err']
print(f"Cantidad de filas que no cumplen la regla: {aux.shape[0]}")

Se visualizan las tuplas que no cumplen con la regla:



,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,regla_pct_ingresos
2,818770008.0,25,5500,3,1.0,HIPOTECA,9600,False,0.57,12.87,1,N,0.0,True,44.0,SOLTERO,ACTIVO,1291.0,F,8256.0,UNIVERSITARIO_COMPLETO,Blue,33.0,5.0,err
75,789973308.0,26,6100,3,2.0,PROPIA,12000,False,0.51,7.51,1,N,0.0,True,35.0,SOLTERO,ACTIVO,1341.0,M,24312.0,UNIVERSITARIO_INCOMPLETO,Blue,24.0,1.0,err
160,718759833.0,23,7750,4,2.0,PROPIA,13920,False,0.56,17.04,1,N,0.0,True,35.0,CASADO,ACTIVO,1058.0,F,32643.0,UNIVERSITARIO_INCOMPLETO,Silver,24.0,2.0,err
460,717392358.0,24,13000,3,6.0,PROPIA,18000,False,0.72,7.88,1,N,0.0,True,50.0,CASADO,ACTIVO,1786.0,M,23412.0,UNIVERSITARIO_COMPLETO,Blue,50.0,1.0,err
469,711864783.0,24,10000,2,0.0,HIPOTECA,18000,False,0.56,12.18,1,N,0.0,True,46.0,SOLTERO,ACTIVO,1199.0,M,2201.0,UNIVERSITARIO_INCOMPLETO,Blue,32.0,1.0,err


Cantidad de filas que no cumplen la regla: 15


Exportacion de los datos modificados

In [28]:
df_creditos.to_csv("../../data/processed/datos_creditos_mc.csv",sep=';',index=False)
df_tarjetas.to_csv("../../data/processed/datos_tarjetas_mc.csv",sep=';',index=False)
df_resultado.to_csv("../../data/processed/datos_integrados_mc.csv",sep=';',index=False)